# Quantum Subspace Expansion (QSE) on $H_2$

This notebook demonstrates the Quantum Subspace Expansion (QSE) algorithm in `qibochem`.
We work with H₂ in the STO-3G basis set.

The workflow is:
1. Build the $H_2$ molecular Hamiltonian.
2. Prepare a reference state using the UCCSD ansatz with an MP2 initial guess of the parameters.
3. Run QSE using spin-conserving single-excitation operators as the expansion basis.
4. Compare the QSE energies against the exact eigenspectrum of the Hamiltonian.

In [1]:
import numpy as np

from qibochem.driver import Molecule
from qibochem.ansatz import ucc_ansatz
from qibochem.selected_ci.qse import QSE, QSEConfig, qse

## 1. System Definition & Hamiltonian

We define the H₂ molecule at equilibrium bond length (0.735 Å) in the STO-3G basis set and run a PySCF calculation to obtain the molecular integrals.

In [2]:
# Define H2 at equilibrium bond length and run PySCF
h2 = Molecule([("H", (0.0, 0.0, 0.0)), ("H", (0.0, 0.0, 0.735))], basis="sto-3g")
h2.run_pyscf()

print(f"Number of spin-orbitals (qubits): {h2.nso}")
print(f"Number of electrons: {h2.nelec}")
print(f"Hartree-Fock energy: {h2.e_hf:.12f} Ha")

Number of spin-orbitals (qubits): 4
Number of electrons: 2
Hartree-Fock energy: -1.116998996754 Ha


## 2. Reference State: UCCSD Ansatz with MP2 Guess

We construct a UCCSD circuit using `qibochem`'s `ucc_ansatz` function. This:
- Starts from the Hartree-Fock reference state $|1100\rangle$
- Generates all single and double excitation operators under Jordan-Wigner mapping
- Initialises the circuit parameters using MP2 amplitudes (a physically motivated initial guess)

These MP2 parameters are already a good approximation to the optimised UCCSD parameters for a small system like H₂.

In [3]:
# Build UCCSD circuit seeded with MP2 amplitudes
circuit = ucc_ansatz(h2, excitation_level="D", use_mp2_guess=True)

print(f"Number of qubits in circuit: {circuit.nqubits}")
print(f"Number of variational parameters: {len(circuit.get_parameters())}")
print(f"MP2 initial parameters: {circuit.get_parameters()}")

Number of qubits in circuit: 4
Number of variational parameters: 12
MP2 initial parameters: [((-0.01799286128463806+0j),), ((0.01799286128463806-0j),), ((0.01799286128463806-0j),), ((0.01799286128463806-0j),), ((-0.01799286128463806+0j),), ((-0.01799286128463806+0j),), ((-0.01799286128463806+0j),), ((0.01799286128463806-0j),), ((-0+0j),), ((-0+0j),), ((-0+0j),), ((-0+0j),)]


## 3. Running QSE

The QSE algorithm expands the variational subspace by applying a set of excitation operators $\{E_a\}$ to the reference state $|\Psi\rangle$ and solving the generalised eigenvalue problem:

$$H_{ab} c = S_{ab} c \varepsilon$$

where $H_{ab} = \langle \Psi | E_a^\dagger H E_b | \Psi \rangle$ and $S_{ab} = \langle \Psi | E_a^\dagger E_b | \Psi \rangle$.

Here we use all spin-conserving one-body excitations ($E_a = a_i^\dagger a_j$ with $i,j$ same spin) as expansion operators.

Setting `n_shots=None` (the default) uses exact statevector simulation.

In [4]:
# Configure and run QSE
# conserve_spin=True generates spin-conserving single excitations + identity
# This mirrors the InQuanto `generate_subspace_singlet_singles()` expansion
config = QSEConfig(
    conserve_spin=True,
    ferm_qubit_map="jw",
    excitation_threshold=1e-9,
    eigenvalue_threshold=1e-9,
    n_shots=None,  # exact statevector simulation
)

result = qse(h2, circuit, config=config)

print(f"QSE energies (Ha): {result.energies}")
print(f"Subspace dimension (before filtering): {result.subspace_dimension}")
print(f"Projected subspace dimension (after overlap filtering): {result.projected_subspace_dimension}")

[Qibo 0.2.20|INFO|2026-02-23 00:10:05]: Using numpy backend on /CPU:0


QSE energies (Ha): [-1.13730604 -0.52461556 -0.16275316  0.49505774]
Subspace dimension (before filtering): 9
Projected subspace dimension (after overlap filtering): 4


## 4. Comparison with Exact Eigenspectrum

Here we compute the exact energies corresponding to the qubit Hamiltonian for reference.

In [5]:
from openfermion.linalg import eigenspectrum

# Compute exact eigenvalues of the full qubit Hamiltonian
qubit_hamiltonian = h2.hamiltonian(ham_type="q", ferm_qubit_map="jw")
all_exact_energies = eigenspectrum(qubit_hamiltonian)

print("All exact eigenvalues (Ha):")
for i, e in enumerate(all_exact_energies):
    print(f"  E[{i}] = {e:.12f} Ha")

All exact eigenvalues (Ha):
  E[0] = -1.137306035753 Ha
  E[1] = -0.536370078554 Ha
  E[2] = -0.536370078554 Ha
  E[3] = -0.524615555364 Ha
  E[4] = -0.524615555364 Ha
  E[5] = -0.524615555364 Ha
  E[6] = -0.440662743309 Ha
  E[7] = -0.440662743309 Ha
  E[8] = -0.162753155796 Ha
  E[9] = 0.248072987168 Ha
  E[10] = 0.248072987168 Ha
  E[11] = 0.366643890342 Ha
  E[12] = 0.366643890342 Ha
  E[13] = 0.495057741618 Ha
  E[14] = 0.719968994449 Ha
  E[15] = 0.934247232868 Ha


Next we print out the energies calculated by QSE.

We also print out the projected subspace dimension before removing near linear-dependent basis states, as well as the dimensions after truncation.

In [ ]:
# The QSE expansion using spin-conserving operators.
n_qse = result.projected_subspace_dimension

print(f"\nSubspace dimension (before filtering): {result.subspace_dimension}")
print(f"Subspace dimension (after filtering): {n_qse}\n")

print("QSE energies (Ha):")
for i in range(n_qse):
    print(f"  QSE E[{i}] = {result.energies[i]:.12f} Ha")


Subspace dimension (before filtering): 9
Subspace dimension (after filtering): 4

QSE energies (Ha):
  QSE E[0] = -1.137306035753 Ha
  QSE E[1] = -0.524615555364 Ha
  QSE E[2] = -0.162753155796 Ha
  QSE E[3] = 0.495057741618 Ha
